In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

from cartopy import crs as ccrs 
import cartopy.feature as cfeature
# import hvplot.xarray

In [ ]:
# Memory check
!free -h

In [ ]:
def test_align_exact(ds1, ds2):
    """
    Test two datasets are exactly aligned
    """
    try:
        ds1_aligned, ds2_aligned = xr.align(ds1, ds2, join="exact")
        print("Aligned.")
        return ds1_aligned, ds2_aligned

    except ValueError as e:
        print(e)
        return None

## LPJ

### Daily NPP

In [ ]:
lpj_path = "/discover/nobackup/projects/GHGC/LPJ_collaborations/NRT_carbon_budget_const_lu/20250521/S2_RESP_ACCLIM/ncdf_outputs"

In [ ]:
lpj_dnpp = f"{lpj_path}/ERA5_S2_RESP_ACCLIM_dnpp.nc"

In [ ]:
ds_dnpp = xr.open_dataset(lpj_dnpp,
                         # chunks="auto"
                         )

### Daily Rh

In [ ]:
lpj_drh = f"{lpj_path}/ERA5_S2_RESP_ACCLIM_drh.nc"

In [ ]:
ds_drh = xr.open_dataset(lpj_drh)
ds_drh

### Combine to calc NEE (NEE = RH - NPP)

In [ ]:
ds_aligned1, ds_aligned2 = test_align_exact(ds_drh, ds_dnpp)

In [ ]:
ds_lpj_combined = xr.merge([ds_drh, ds_dnpp])

In [ ]:
ds_lpj_combined

## MiCASA

In [ ]:
micasa_path = "micasa_virtualized/vstore.parquet"

In [ ]:
ds_mi = xr.open_dataset(f"reference::{micasa_path}", 
                        engine="zarr",
                        consolidated=False,
                        )
ds_mi

In [ ]:
ds_mi["NPP"]

In [ ]:
ds_mi_chunk = ds_mi.chunk({'time': 30, 'lat': 900, 'lon': 1800})
ds_mi_chunk

## Align MiCASA and LPJ

In [ ]:
# Micasa starts at Jan 2001
ds_lpj_sel = ds_lpj_combined.sel(time=slice("2001", None))
ds_lpj_sel

##### 6 days missing? LPJ does NOT include leap days????

In [ ]:
# Downsample to match LPJ
ds_mi_downsample = ds_mi_chunk.coarsen(lat=5,lon=5, boundary="trim").mean() # Downsampling (5x5 aggregation since 0.5°/0.1° = 5)
ds_mi_downsample

In [ ]:
test_align_exact(ds_lpj_sel, ds_mi_downsample) # Leap days still not aligned

In [ ]:
# Chunking
chunk_config = {'time': 365, 'lat': 900, 'lon': 1800}

In [ ]:
# Drop leap days to match LPJ and drop unneeded vars
leap_day_mask = ~((ds_mi_downsample.time.dt.month == 2) & (ds_mi_downsample.time.dt.day == 29))
ds_mi_noleap = ds_mi_downsample.sel(time=leap_day_mask)
# Rechunk after remove leap days
ds_mi_sel = ds_mi_noleap.chunk(chunk_config)[["NEE", "NPP", "Rh"]]
ds_mi_sel

In [ ]:
# # Change var names for consistency with micasa
ds_lpj_sel = ds_lpj_sel.rename_dims({"latitude": "lat", "longitude": "lon"})
ds_lpj_sel = ds_lpj_sel.rename_vars({"latitude": "lat", "longitude": "lon"})

# print(ds_lpj_sel.lon.values[:5], ds_lpj_sel.lon.values[-5:])
# print(ds_mi_sel.lon.values[:5], ds_mi_sel.lon.values[-5:])

test_align_exact(ds_lpj_sel, ds_mi_sel) # After renaming they aren't aligned?? 
# # Maybe because before they were diff dimensions so they didn't show up as not aligned

In [ ]:
print(np.array_equal(ds_lpj_sel.lat.values, ds_mi_sel.lat.values))
print(np.array_equal(ds_lpj_sel.lon.values, ds_mi_sel.lon.values))

In [ ]:
print(ds_mi_sel.lat.values[:5], ds_lpj_sel.lat.values[:5])
type(ds_mi_sel.lat.values[0]), type(ds_lpj_sel.lat.values[0])

In [ ]:
# ds_lpj_fix = ds_lpj_sel.copy()
# ds_lpj_fix.coords['lat'] = ds_lpj_sel.lat.astype(ds_mi_sel.lat.dtype)
# ds_lpj_fix.coords['lon'] = ds_lpj_sel.lon.astype(ds_mi_sel.lon.dtype)

In [ ]:
# print(np.array_equal(ds_lpj_fix.lat.values, ds_mi_sel.lat.values))

In [ ]:
# test_align_exact(ds_lpj_fix, ds_mi_sel) # Still not aligned

In [ ]:
# test = (ds_mi_sel.lon.values == ds_lpj_fix.lon.values)
# test[:5] # Still not matching

In [ ]:
# Force Micasa to fit lpj???
ds_mi_sel = ds_mi_sel.reindex(lat=ds_lpj_sel.lat, lon=ds_lpj_sel.lon, method='nearest')
ds_lpj_align, ds_mi_align = test_align_exact(ds_lpj_sel, ds_mi_sel)

In [ ]:
ds_lpj_align = ds_lpj_align.chunk(chunk_config)
ds_lpj_align

## Comparisons

In [ ]:
print(ds_lpj_align["dnpp"].attrs, ds_mi_align["NPP"].attrs, sep="\n\n")

In [ ]:
result = ds_lpj_align["dnpp"]/86400 - ds_mi_align["NPP"] # Conversion? did i do that right
result

In [ ]:
# result_loaded.hvplot(
#     x="lon", y="lat",
#     groupby="time", 
#     cmap="coolwarm",     
#     clim = (-1.5e-7, 1.5e-7),
#     height=400, width=800,
#     widget_location="bottom",
#     title="NPP Difference (LPJ-MiCASA)"
# )

### Test with 2024 only

In [ ]:
# result_sel = result.sel(time=slice("2022", "2024"))
results_subset = result.sel(time="2024")
results_rechunked = results_subset.chunk({'time': 90, 'lat': -1, 'lon': -1})
results_rechunked

In [ ]:
# %%time
# This still takes the same  amount of time as the whole dataset?
# results_sub = results_rechunked.sel(lat=slice(25, 60), lon=slice(-125, -60)).compute()

# This is the same time as just computing everything first and then grouping
# seasonal_means = results_rechunked.groupby(results_rechunked.time.dt.season).mean().compute()

In [ ]:
%%time
results_compute = results_rechunked.compute() 

In [ ]:
# Mask NaNs over ocean, this still didn't fix my nonnull.any issue
mask = results_compute.notnull().any(dim=["time"])
results_masked = results_compute.where(mask,drop=True)
results_masked

In [ ]:
results_masked = results_masked.astype("float64")

### Means

In [ ]:
seasonal_means = results_masked.groupby(results_masked.time.dt.season).mean()
seasonal_means

#### Plot mean

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
plot = seasonal_means.sel(season='DJF').plot.pcolormesh(ax=ax, 
                                                        transform=ccrs.PlateCarree(),
                                                        cmap="RdBu",
                                                       add_colorbar=False,)
cb = plt.colorbar(plot,orientation='horizontal', shrink=0.8)


In [ ]:
for season in seasonal_means.season.values:
    min = seasonal_means.sel(season=season).min().values
    max = seasonal_means.sel(season=season).max().values
    
    print(f"{season}: {min:.2e}, {max:.2e}")

In [ ]:
seasonal_means.min().values, seasonal_means.max().values

In [ ]:
seasonal_means.var().values

In [ ]:
%matplotlib inline
proj = ccrs.PlateCarree()
fig, axs = plt.subplots(2,2, figsize=(12, 8), subplot_kw= {'projection': proj});
for ax, season in zip(axs.flat, seasonal_means.season.values):
    # print(i, season)
    plot = seasonal_means.sel(season=season).plot.pcolormesh(ax=ax, 
                                                        transform=ccrs.PlateCarree(),
                                                             vmin=-9e-8,vmax=9e-8,
                                                             
                                                             cmap="RdBu",
                                                       add_colorbar=False,)
    cb = plt.colorbar(plot,orientation='horizontal', shrink=0.8, pad=0.05, 
                      # label="LPJ - MiCASA\nMean NPP difference (kg C m-2 s-1)\n2024 avg"
                              label="Mean difference NPP (kg C m-2 s-1)\nLPJ-EOSIM — MiCASA",

                     )
    

### Test output files

In [ ]:
# output_dir = "tests"
# import os
# os.makedirs(output_dir, exist_ok=True)

for season in seasonal_means.season.values:
    fig, ax = plt.subplots(1,1, figsize=(16, 8), subplot_kw= {'projection': proj})
    
    # print(i, season)
    plot = seasonal_means.sel(season=season).plot.pcolormesh(ax=ax, 
                                                        transform=ccrs.PlateCarree(),
                                                             vmin=-9e-8,vmax=9e-8,
                                                             cmap="RdBu",
                                                       add_colorbar=False,);
    ax.set_title(f"{season}");
    cb = plt.colorbar(plot,orientation='horizontal', shrink=0.8, pad=0.05, 
                    label="LPJ-EOSIM — MiCASA (kg C m-2 s-1)\n2024",

                     );
    fig.suptitle("Mean model difference, NPP", x=0.5, y=.92,fontsize=15)
    plt.show()
    break
    # output_filename = f"NPPDiff_{season}.png"
    # output_path = os.path.join(output_dir, output_filename)
    # fig.savefig(output_path)

### Variance

In [ ]:
# This spits warnings about NaN values but it still should wokr
seasonal_var = results_masked.groupby(results_masked.time.dt.season).var(dim="time",skipna=True,ddof=0)

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
plot = seasonal_var.sel(season='JJA').plot.pcolormesh(ax=ax, 
                                                        transform=ccrs.PlateCarree(),
                                                        cmap="Reds",
                                                       add_colorbar=False,)
cb = plt.colorbar(plot,orientation='horizontal', shrink=0.8)